In [ ]:
# Custom Exceptions and Real-World Applications

"""
Custom exceptions help create more specific error handling for your applications.
They make code more readable and allow for better error management.
"""

# Basic Custom Exception
class CustomError(Exception):
    """Base class for custom exceptions"""
    pass

# Example 1: Banking System with Custom Exceptions
class BankingError(Exception):
    """Base exception for banking operations"""
    def __init__(self, message, error_code=None):
        super().__init__(message)
        self.error_code = error_code

class InsufficientFundsError(BankingError):
    """Raised when account has insufficient funds"""
    def __init__(self, balance, requested_amount):
        self.balance = balance
        self.requested_amount = requested_amount
        message = f"Insufficient funds: Balance ${balance:.2f}, Requested ${requested_amount:.2f}"
        super().__init__(message, "INSUFFICIENT_FUNDS")

class InvalidAccountError(BankingError):
    """Raised when account number is invalid"""
    def __init__(self, account_number):
        self.account_number = account_number
        message = f"Invalid account number: {account_number}"
        super().__init__(message, "INVALID_ACCOUNT")

class AccountFrozenError(BankingError):
    """Raised when trying to operate on frozen account"""
    def __init__(self, account_number, reason="Security reasons"):
        self.account_number = account_number
        self.reason = reason
        message = f"Account {account_number} is frozen: {reason}"
        super().__init__(message, "ACCOUNT_FROZEN")

class BankAccount:
    """Bank account class with custom exception handling"""
    
    def __init__(self, account_number, initial_balance=0, is_frozen=False):
        self.account_number = account_number
        self.balance = initial_balance
        self.is_frozen = is_frozen
        self.transaction_history = []
    
    def validate_account(self):
        """Validate account status"""
        if len(str(self.account_number)) < 5:
            raise InvalidAccountError(self.account_number)
        
        if self.is_frozen:
            raise AccountFrozenError(self.account_number)
    
    def withdraw(self, amount):
        """Withdraw money from account"""
        self.validate_account()
        
        if amount <= 0:
            raise ValueError("Withdrawal amount must be positive")
        
        if amount > self.balance:
            raise InsufficientFundsError(self.balance, amount)
        
        self.balance -= amount
        self.transaction_history.append(f"Withdrawal: -${amount:.2f}")
        return self.balance
    
    def deposit(self, amount):
        """Deposit money to account"""
        self.validate_account()
        
        if amount <= 0:
            raise ValueError("Deposit amount must be positive")
        
        self.balance += amount
        self.transaction_history.append(f"Deposit: +${amount:.2f}")
        return self.balance

print("=== Banking System Example ===")

# Create bank accounts
account1 = BankAccount("12345", 1000.00)
account2 = BankAccount("123", 500.00)  # Invalid account number
account3 = BankAccount("67890", 2000.00, is_frozen=True)

# Test normal operations
try:
    account1.deposit(500)
    print(f"Account {account1.account_number} balance: ${account1.balance:.2f}")
    
    account1.withdraw(200)
    print(f"After withdrawal: ${account1.balance:.2f}")
    
except BankingError as e:
    print(f"Banking Error: {e} (Code: {e.error_code})")
except ValueError as e:
    print(f"Value Error: {e}")

# Test insufficient funds
try:
    account1.withdraw(2000)  # Should fail
except InsufficientFundsError as e:
    print(f"\n❌ {e}")
    print(f"Available: ${e.balance:.2f}, Requested: ${e.requested_amount:.2f}")

# Test invalid account
try:
    account2.deposit(100)  # Should fail due to invalid account
except InvalidAccountError as e:
    print(f"\n❌ {e}")
    print(f"Invalid account number: {e.account_number}")

# Test frozen account
try:
    account3.withdraw(100)  # Should fail due to frozen account
except AccountFrozenError as e:
    print(f"\n❌ {e}")
    print(f"Reason: {e.reason}")

# Example 2: E-commerce System with Custom Exceptions
class ECommerceError(Exception):
    """Base exception for e-commerce operations"""
    pass

class ProductNotFoundError(ECommerceError):
    """Raised when product is not found"""
    def __init__(self, product_id):
        self.product_id = product_id
        super().__init__(f"Product with ID {product_id} not found")

class OutOfStockError(ECommerceError):
    """Raised when product is out of stock"""
    def __init__(self, product_name, requested_qty, available_qty):
        self.product_name = product_name
        self.requested_qty = requested_qty
        self.available_qty = available_qty
        super().__init__(
            f"{product_name}: Requested {requested_qty}, Available {available_qty}"
        )

class InvalidDiscountError(ECommerceError):
    """Raised when discount is invalid"""
    def __init__(self, discount_code):
        self.discount_code = discount_code
        super().__init__(f"Invalid discount code: {discount_code}")

class ShippingError(ECommerceError):
    """Raised when shipping is not available"""
    def __init__(self, address, reason="Address not serviceable"):
        self.address = address
        self.reason = reason
        super().__init__(f"Shipping error for {address}: {reason}")

class Product:
    """Product class"""
    def __init__(self, product_id, name, price, stock):
        self.product_id = product_id
        self.name = name
        self.price = price
        self.stock = stock

class ShoppingCart:
    """Shopping cart with custom exception handling"""
    
    def __init__(self):
        self.items = {}
        self.products_db = {
            "P001": Product("P001", "Laptop", 999.99, 5),
            "P002": Product("P002", "Mouse", 29.99, 0),  # Out of stock
            "P003": Product("P003", "Keyboard", 79.99, 10)
        }
        self.valid_discounts = {"SAVE10": 0.1, "WELCOME20": 0.2}
    
    def add_item(self, product_id, quantity):
        """Add item to cart"""
        if product_id not in self.products_db:
            raise ProductNotFoundError(product_id)
        
        product = self.products_db[product_id]
        
        if product.stock < quantity:
            raise OutOfStockError(product.name, quantity, product.stock)
        
        if product_id in self.items:
            self.items[product_id] += quantity
        else:
            self.items[product_id] = quantity
        
        # Update stock
        product.stock -= quantity
        
        return f"Added {quantity} x {product.name} to cart"
    
    def apply_discount(self, discount_code):
        """Apply discount to cart"""
        if discount_code not in self.valid_discounts:
            raise InvalidDiscountError(discount_code)
        
        return self.valid_discounts[discount_code]
    
    def calculate_total(self, discount_code=None):
        """Calculate cart total"""
        total = 0
        for product_id, quantity in self.items.items():
            product = self.products_db[product_id]
            total += product.price * quantity
        
        if discount_code:
            discount = self.apply_discount(discount_code)
            total *= (1 - discount)
        
        return total
    
    def checkout(self, shipping_address, discount_code=None):
        """Process checkout"""
        if not shipping_address or len(shipping_address.strip()) < 10:
            raise ShippingError(shipping_address, "Invalid shipping address")
        
        # Simulate shipping validation
        restricted_areas = ["restricted_area", "no_delivery_zone"]
        if any(area in shipping_address.lower() for area in restricted_areas):
            raise ShippingError(shipping_address, "Delivery not available in this area")
        
        total = self.calculate_total(discount_code)
        return f"Order processed successfully! Total: ${total:.2f}"

print("\n=== E-commerce System Example ===")

cart = ShoppingCart()

# Test normal operations
try:
    print(cart.add_item("P001", 2))  # Add laptop
    print(cart.add_item("P003", 1))  # Add keyboard
    
    total = cart.calculate_total("SAVE10")
    print(f"Cart total with discount: ${total:.2f}")
    
    result = cart.checkout("123 Main St, New York, NY 10001")
    print(result)

except ECommerceError as e:
    print(f"E-commerce Error: {e}")

# Test product not found
try:
    cart.add_item("P999", 1)  # Invalid product
except ProductNotFoundError as e:
    print(f"\n❌ Product Error: {e}")
    print(f"Product ID: {e.product_id}")

# Test out of stock
try:
    cart.add_item("P002", 1)  # Out of stock item
except OutOfStockError as e:
    print(f"\n❌ Stock Error: {e}")
    print(f"Product: {e.product_name}")
    print(f"Requested: {e.requested_qty}, Available: {e.available_qty}")

# Test invalid discount
try:
    cart.calculate_total("INVALID_CODE")
except InvalidDiscountError as e:
    print(f"\n❌ Discount Error: {e}")
    print(f"Code: {e.discount_code}")

# Test shipping error
try:
    cart.checkout("restricted_area")
except ShippingError as e:
    print(f"\n❌ Shipping Error: {e}")
    print(f"Address: {e.address}")
    print(f"Reason: {e.reason}")


In [ ]:

# Example 3: Advanced Exception Handling with Context Managers
class DatabaseConnectionError(Exception):
    """Raised when database connection fails"""
    pass

class DatabaseQueryError(Exception):
    """Raised when database query fails"""
    def __init__(self, query, original_error):
        self.query = query
        self.original_error = original_error
        super().__init__(f"Query failed: {query} - Error: {original_error}")

class DatabaseConnection:
    """Simulated database connection with custom exceptions"""
    
    def __init__(self, connection_string):
        self.connection_string = connection_string
        self.is_connected = False
        self.transaction_active = False
    
    def connect(self):
        """Connect to database"""
        if "invalid" in self.connection_string:
            raise DatabaseConnectionError("Failed to connect to database")
        self.is_connected = True
        print("✅ Connected to database")
    
    def disconnect(self):
        """Disconnect from database"""
        self.is_connected = False
        print("✅ Disconnected from database")
    
    def begin_transaction(self):
        """Begin database transaction"""
        if not self.is_connected:
            raise DatabaseConnectionError("Not connected to database")
        self.transaction_active = True
        print("✅ Transaction started")
    
    def commit_transaction(self):
        """Commit database transaction"""
        if not self.transaction_active:
            raise DatabaseQueryError("COMMIT", "No active transaction")
        self.transaction_active = False
        print("✅ Transaction committed")
    
    def rollback_transaction(self):
        """Rollback database transaction"""
        if self.transaction_active:
            self.transaction_active = False
            print("↩️ Transaction rolled back")
    
    def execute_query(self, query):
        """Execute database query"""
        if not self.is_connected:
            raise DatabaseConnectionError("Not connected to database")
        
        # Simulate query errors
        if "DROP" in query.upper():
            raise DatabaseQueryError(query, "DROP operations not allowed")
        
        if "INVALID_SYNTAX" in query.upper():
            raise DatabaseQueryError(query, "Syntax error in query")
        
        return f"Query executed: {query}"
    
    def __enter__(self):
        """Context manager entry"""
        self.connect()
        self.begin_transaction()
        return self
    
    def __exit__(self, exc_type, exc_value, traceback):
        """Context manager exit"""
        try:
            if exc_type is None:
                self.commit_transaction()
            else:
                self.rollback_transaction()
                print(f"❌ Exception occurred: {exc_value}")
        finally:
            self.disconnect()
        
        # Don't suppress exceptions
        return False

print("\n=== Database Exception Handling Example ===")

# Test successful database operations
try:
    with DatabaseConnection("postgresql://localhost:5432/mydb") as db:
        result1 = db.execute_query("SELECT * FROM users")
        print(result1)
        
        result2 = db.execute_query("INSERT INTO users (name) VALUES ('John')")
        print(result2)
        
        print("✅ All operations completed successfully")

except (DatabaseConnectionError, DatabaseQueryError) as e:
    print(f"Database Error: {e}")

# Test database error with rollback
try:
    with DatabaseConnection("postgresql://localhost:5432/mydb") as db:
        result1 = db.execute_query("SELECT * FROM users")
        print(result1)
        
        # This will cause an error and trigger rollback
        result2 = db.execute_query("DROP TABLE users")
        print(result2)

except DatabaseQueryError as e:
    print(f"\n❌ Database Query Error: {e}")
    print(f"Failed query: {e.query}")
    print(f"Original error: {e.original_error}")


In [1]:

# Example 4: Exception Chaining and Custom Exception Hierarchy
class ValidationError(Exception):
    """Base validation exception"""
    pass

class EmailValidationError(ValidationError):
    """Email validation specific error"""
    pass

class PasswordValidationError(ValidationError):
    """Password validation specific error"""
    pass

class AgeValidationError(ValidationError):
    """Age validation specific error"""
    pass

class UserValidator:
    """User input validator with chained exceptions"""
    
    @staticmethod
    def validate_email(email):
        """Validate email format"""
        if not email or "@" not in email or "." not in email:
            raise EmailValidationError(f"Invalid email format: {email}")
        return True
    
    @staticmethod
    def validate_password(password):
        """Validate password strength"""
        if len(password) < 8:
            raise PasswordValidationError("Password must be at least 8 characters long")
        
        if not any(c.isupper() for c in password):
            raise PasswordValidationError("Password must contain at least one uppercase letter")
        
        if not any(c.isdigit() for c in password):
            raise PasswordValidationError("Password must contain at least one digit")
        
        return True
    
    @staticmethod
    def validate_age(age):
        """Validate age range"""
        try:
            age_int = int(age)
            if age_int < 0 or age_int > 150:
                raise AgeValidationError(f"Age must be between 0 and 150, got {age_int}")
            return True
        except ValueError as e:
            # Chain the original exception
            raise AgeValidationError(f"Age must be a valid number, got '{age}'") from e
    
    @classmethod
    def validate_user_data(cls, email, password, age):
        """Validate all user data"""
        errors = []
        
        try:
            cls.validate_email(email)
        except EmailValidationError as e:
            errors.append(str(e))
        
        try:
            cls.validate_password(password)
        except PasswordValidationError as e:
            errors.append(str(e))
        
        try:
            cls.validate_age(age)
        except AgeValidationError as e:
            errors.append(str(e))
        
        if errors:
            raise ValidationError(f"Validation failed: {'; '.join(errors)}")
        
        return True

print("\n=== User Validation Example ===")

# Test valid user data
try:
    UserValidator.validate_user_data("user@example.com", "SecurePass123", "25")
    print("✅ User data validation passed")
except ValidationError as e:
    print(f"❌ Validation Error: {e}")

# Test invalid user data
try:
    UserValidator.validate_user_data("invalid-email", "weak", "abc")
    print("✅ User data validation passed")
except ValidationError as e:
    print(f"❌ Validation Error: {e}")

# Test exception chaining
try:
    UserValidator.validate_age("not_a_number")
except AgeValidationError as e:
    print(f"\n❌ Age Validation Error: {e}")
    if e.__cause__:
        print(f"Original cause: {type(e.__cause__).__name__}: {e.__cause__}")

print("\n=== Exception Handling Best Practices ===")
print("1. Create specific exception classes for different error types")
print("2. Include relevant context in exception messages")
print("3. Use exception chaining with 'raise ... from ...' when appropriate")
print("4. Implement proper cleanup in exception handlers")
print("5. Use context managers for resource management")
print("6. Log exceptions appropriately for debugging")
print("7. Don't catch exceptions you can't handle meaningfully")


=== User Validation Example ===
✅ User data validation passed
❌ Validation Error: Validation failed: Invalid email format: invalid-email; Password must be at least 8 characters long; Age must be a valid number, got 'abc'

❌ Age Validation Error: Age must be a valid number, got 'not_a_number'
Original cause: ValueError: invalid literal for int() with base 10: 'not_a_number'

=== Exception Handling Best Practices ===
1. Create specific exception classes for different error types
2. Include relevant context in exception messages
3. Use exception chaining with 'raise ... from ...' when appropriate
4. Implement proper cleanup in exception handlers
5. Use context managers for resource management
6. Log exceptions appropriately for debugging
7. Don't catch exceptions you can't handle meaningfully
